In [1]:
from datetime import datetime
import pytz
import os
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from model import Net,run_check_net
from dataset import read_one_data, read_one_truth
from czii_helper import dotdict

#分子の種類
MOLECULES = ['apo-ferritin', 'beta-amylase', 'beta-galactosidase', 'ribosome', 'thyroglobulin', 'virus-like-particle']
value_map = {m: i+1 for i, m in enumerate(MOLECULES)}

# ログ出力
print('LOGGING TIME OF START:', datetime.strftime(datetime.now(pytz.timezone('Asia/Singapore')), "%Y-%m-%d %H:%M:%S"))

# データセットディレクトリ
DATA_KAGGLE_DIR = '../input/czii-cryo-et-object-identification'

# モード設定
MODE = 'local'
if MODE == 'local':
    valid_dir = f'{DATA_KAGGLE_DIR}/train'
    valid_id = ['TS_5_4', 'TS_73_6', 'TS_99_9']
else:
    valid_dir = f'{DATA_KAGGLE_DIR}/test'
    valid_id = [os.path.basename(path) for path in os.listdir(f'{valid_dir}/static/ExperimentRuns')]

# 学習ディレクトリ

train_dir = f'{DATA_KAGGLE_DIR}/train'
train_id = [os.path.basename(path) for path in os.listdir(f'{train_dir}/static/ExperimentRuns')]
train_id = [id for id in train_id if id not in valid_id]
print('TRAIN ID:', train_id)

# モデル設定
cfg = dotdict({
    'arch': 'resnet34d',
    'threshold': {
        'apo-ferritin': 0.05,
        'beta-amylase': 0.05,
        'beta-galactosidase': 0.05,
        'ribosome': 0.05,
        'thyroglobulin': 0.05,
        'virus-like-particle': 0.05,
    },
})
print('MODE:', MODE)
print('SETTING OK!')


/home2/y2023/m2311203/.conda/envs/ultralytics-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LOGGING TIME OF START: 2025-01-05 20:07:21
TRAIN ID: ['TS_69_2', 'TS_6_4', 'TS_6_6', 'TS_86_3']
MODE: local
SETTING OK!


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# マスクデータ生成（仮データ例として利用可能）
num_slice = 100
mask = np.zeros((num_slice, 640, 640), dtype=np.float32)

# 仮データの設定
mask[50, 200:220, 300:320] = 1  # 値1のボクセル
mask[70, 100:120, 400:420] = 2  # 値2のボクセル
mask[30, 300:320, 200:220] = 3  # 値3のボクセル

### 1. スライスごとの2D可視化 ###
def visualize_slices(mask, slice_indices):
    """
    マスクの特定のスライスを可視化
    """
    plt.figure(figsize=(12, len(slice_indices) * 4))
    for i, slice_idx in enumerate(slice_indices):
        plt.subplot(1, len(slice_indices), i + 1)
        plt.imshow(mask[slice_idx], cmap='viridis', origin='lower')
        plt.colorbar(label="Values")
        plt.title(f"Slice {slice_idx}")
        plt.xlabel("X")
        plt.ylabel("Y")
    plt.tight_layout()
    plt.show()

### 2. 3D可視化 ###
def visualize_3d(mask):
    """
    マスクの3D可視化
    """
    # ボクセルの座標を取得
    z, y, x = np.nonzero(mask)

    # 色の設定（値に基づいて色を変える）
    colors = np.zeros((len(z), 4))  # RGBA
    for i, value in enumerate(mask[z, y, x]):
        if value == 1:  # 値1は赤
            colors[i] = [1, 0, 0, 0.8]
        elif value == 2:  # 値2は緑
            colors[i] = [0, 1, 0, 0.8]
        elif value == 3:  # 値3は青
            colors[i] = [0, 0, 1, 0.8]

    # プロット
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z, c=colors, marker='o', s=10)

    # 軸ラベル
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title("3D Visualization of Mask")
    plt.show()



In [3]:
def cutout_volume(mask_dict,z_min,num_slice):
    result = {}
    for key, array in mask_dict.items():
        # zが範囲内にある行を抽出
        filtered = array[(array[:, 2] >= z_min*10) & (array[:, 2] <= (z_min*10)+num_slice* 10)]
        if filtered.size > 0:  # 抽出結果が空でない場合のみ
            result[key] = filtered
    return result

In [4]:
slice_id = 0
num_slice = 32
D = 184
num_depth = len(list(range(0, D - num_slice, num_slice//2)) + [D - num_slice]) 

slice_volume = np.zeros((len(train_id)*num_depth, num_slice, 640, 640), dtype=np.float32)
mask_volume = np.zeros((len(train_id)*num_depth, num_slice, 640, 640), dtype=np.float32)

for i,id in enumerate(train_id):
    print(i, id, '---------------')
    volume = read_one_data(id, static_dir=f'{valid_dir}/static/ExperimentRuns')
    mask_dict = read_one_truth(id, f'{valid_dir}/overlay/ExperimentRuns')
    D, H, W = volume.shape
    print(D, H, W)

    pad_volume = np.pad(volume, [[0, 0], [0, 640 - H], [0, 640 - W]], mode='constant', constant_values=0)

    zz = list(range(0, D - num_slice, num_slice//2)) + [D - num_slice]
    for z in zz:
        slice_volume[slice_id] = pad_volume[z:z+num_slice]
        data = cutout_volume(mask_dict, z, num_slice)
        mask = np.zeros((num_slice, 640, 640), dtype=np.float32)
        for key, points in data.items():
            value = value_map[key]
            for point in points:
                x, y, z = point
                z = z - 640
                # インデックスに変換（スケールダウン）
                x_idx = int(x * 0.1)
                y_idx = int(y * 0.1)
                z_idx = int(z * 0.1)
                # インデックスがマスクの範囲内であれば値を設定
                if 0 <= x_idx < mask.shape[2] and 0 <= y_idx < mask.shape[1] and 0 <= z_idx < mask.shape[0]:
                    mask[z_idx, y_idx, x_idx] = value
        mask_volume[slice_id] = mask
        if np.sum(mask) > 0:
            slice_id += 1

print(slice_id)

slice_volume = slice_volume[:slice_id]
mask_volume = mask_volume[:slice_id]

0 TS_69_2 ---------------
184 630 630
1 TS_6_4 ---------------
184 630 630
2 TS_6_6 ---------------
184 630 630
3 TS_86_3 ---------------
184 630 630
16


In [10]:
train_loader
    

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from model import Net  # Assuming "Net" is defined in model.py

# Example dataset class (replace with your actual dataset logic)
class ExampleDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'image': self.data[idx],
            'mask': self.labels[idx]
        }

train_dataset = ExampleDataset(slice_volume, mask_volume)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Model Initialization
model = Net(pretrained=True, cfg=None).cuda()

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0

        for batch_idx, batch in enumerate(dataloader):
            # Move data to GPU
            images = batch['image'].cuda()
            masks = batch['mask'].cuda()

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model({'image': images})
            loss = criterion(outputs['particle'], masks)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Update loss
            running_loss += loss.item()

            if batch_idx % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {running_loss/len(dataloader):.4f}")

    print("Training complete.")

# Start training
train_model(model, train_loader, criterion, optimizer, num_epochs=10)


KeyError: 'mask'